In [41]:
#!pip install spacy==3.0.6

In [42]:
#!pip install -U spacy-nightly --pre

In [43]:
#!pip install -U pip setuptools wheel

In [44]:
#!python -m spacy download en_core_web_trf

In [45]:
#!pip install -U spacy transformers

In [46]:
#!pip install spacy-transformers

In [47]:
#!pip install PyPDF2

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/TA_project/progetto_TA/spacytransformers

/content/drive/MyDrive/TA_project/progetto_TA/spacytransformers


In [7]:
import random

from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
#from spacy.training.example import Example
from rel_pipe import make_relation_extractor, score_relations
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [8]:
nlp2 = spacy.load("/content/drive/MyDrive/TA_project/progetto_TA/spacytransformers/model-best")

/usr/local/lib/python3.7/dist-packages/spacy/util.py:833: UserWarning: [W095] Model 'it_pipeline' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [9]:
type(nlp2)

spacy.lang.it.Italian

In [10]:
import spacy.cli

spacy.cli.download('it_core_news_lg')

✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')


In [11]:
nlp_it = spacy.load('it_core_news_lg')

In [12]:
type(nlp_it)

spacy.lang.it.Italian

## Test model from http://www.italgiure.giustizia.it/sncass/ judgements

In [24]:
import PyPDF2

In [25]:
import pandas as pd

In [14]:
import re

def clean_jud(text):
    text = text.lower()
    text = text.replace("'"," ")
    #text = text[text.find(' 1.')+3: text.find(' 1.') + 800]
    text = re.sub('art. \d', '',text)
    text = re.sub('artt. \d', '',text)
    text = re.sub("d.p.r. \d", '',text)
    text = re.sub("n. \d", '',text)
    text = re.sub("n. dep \d", '',text)
    text = re.sub('ex \d','',text)
    text = re.sub('comma \d', '',text)

    
    text = text.replace('n.','')
    text = text.replace('cost.','')
    text = text.replace('sent.', '')
    text = text.replace('num.', '')
    text = text.replace('sez.', '')
    text = text.replace('p. e p.', '')
    text = text.replace("d.lgs.", '')
    text = text.replace("cod. pen.", '')
    text = text.replace("cod.proc.pe", '')
    text = text.replace('m.a.e', '')
    text = text.replace('lett.', '')
    
    

    
    text = re.sub(' +', ' ',text)
    return text

In [15]:
def pdf2str(pdf_link):
    !wget -q '{pdf_link}'  -O 'sentenza.pdf'
    pdfFileObj = open('sentenza.pdf', 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    text = ''
    for p in range(pdfReader.numPages):
        text +=  pdfReader.getPage(p).extractText()
    !rm 'sentenza.pdf'
    return text

In [36]:
def get_imputati(pdf_link, top_k=3):
    text = pdf2str(pdf_link)
    text = clean_jud(text)

    sentenza = nlp2(nlp_it(text))
    imputati = []
    for value, rel_dict in sentenza._.rel.items():
        for s in sentenza.sents:
            for e in s.ents:
                for b in s.ents:
                    if e.start == value[0] and b.start == value[1]:
                        if rel_dict['DIFENDE'] > rel_dict['GIUDICA']:
                            imputati.append((b.text[0]+'*****'+b.text[-1], rel_dict['DIFENDE'],b.text))   
                        
                        
                        else:
                            imputati.append((b.text[0]+'*****'+b.text[-1],  rel_dict['GIUDICA'],b.text))

    imputati.sort(key= lambda x: x[1],reverse=True)
    imp_name = set()
    new_imputati = []
    for t in imputati:
        if t[0] not in imp_name:
            new_imputati.append(t)
            imp_name.add(t[0])
    top_k = int(top_k)
    if top_k > len(new_imputati):
        top_k = len(new_imputati)
        
    return pd.DataFrame(new_imputati[:top_k], columns =['defendants','probs', 'completename'])


In [37]:
# from cassazione website get a pdf link
pdf_link = "http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211220/snpen@s70@a2021@n46547@tS.clean.pdf"

In [38]:
df = get_imputati(pdf_link,3)
df

,defendants,probs,completename
0,g*****i,0.994053,giovanni carparelli
1,p*****e,0.506994,pe
2,2*****.,0.042402,26/7/2021.


In [39]:
# text output
text = pdf2str(pdf_link)
text = clean_jud(text)
text

'sentenza sul ricorso proposto da: carparelli giovanni nato a taranto il 02/12/1981 avverso la sentenza del 12/02/2021 del gip tribunale di lecce dalslavviso- udita la relazione svolta dal consigliere giovanni liberati; penale 7 46547 an021presidente: ramacci lucarelatore: liberati giovannidata udienza: 26/11/2021ritenuto in fatto con la sentenza indicata in epigrafe il giudice per le indagini preliminari del tribunale di lecce ha applicato a giovanni carparelli, su sua richiesta ai sensi dell 44 cod. proc. pe, la pena di 1 anno e 6 mesi di reclusione e 2.000 euro di multa, in relazione al reato di cui all 3, , 09/90. avverso tale sentenza l imputato ha proposto ricorso per cassazione, lamentando l illogicità della motivazione e la violazione dell 92 cod. proc. pe, non essendo stato considerato che la sostanza stupefacente acquistata era destinata al consumo personale o di gruppo, e la mancata applicazione della causa di esclusione della punibilità per la particolare tenuità del fatto 

In [40]:
imp = df.iloc[0]['completename']
    
text = text[text.find(imp)-200: text.find(imp) + 200]
text = text.replace(imp, imp[0]+ '********' + imp[-1])
print(text)

nte: ramacci lucarelatore: liberati giovannidata udienza: 26/11/2021ritenuto in fatto con la sentenza indicata in epigrafe il giudice per le indagini preliminari del tribunale di lecce ha applicato a g********i, su sua richiesta ai sensi dell 44 cod. proc. pe, la pena di 1 anno e 6 mesi di reclusione e 2.000 euro di multa, in relazione al reato di cui all 3, , 09/90. avverso tale sentenza
